## Example for batch processing of images
How to select RGB bands from a set of multiband images in a folder

In [1]:
import numpy as np
import os
import skimage.io as skio
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import imageio

#### Functions

In [2]:
def select_rgb_bands_from_multiband(f_in, f_out, rgb_bands = [3, 2, 1], 
                                    crop_min_perc = 0.5, crop_max_perc = 99.5):
    ## Read input multiband image
    mat_in = skio.imread(f_in, plugin="tifffile")
    
    ## Select bands
    mat_out = mat_in[:,:, [3, 2, 1]]
    
    ## Scale img in each band (using percentiles, as it's more robust to outliers)
    for i in [0, 1, 2]:
        scmin = np.percentile(mat_out[:, :, i], crop_min_perc)
        scmax = np.percentile(mat_out[:, :, i], crop_max_perc)
        delta = scmax - scmin
        mat_out[:, :, i] = ((mat_out[:, :, i] - scmin) / delta * 255).astype(int)
        
    ## Crop values that are out of range (defined by percentiles)
    mat_out[mat_out < 0] = 0
    mat_out[mat_out > 255] = 255

    ## Create output folder
    try:
        out_dir = os.path.dirname(f_out)
        if os.path.exists(out_dir) == False :
            os.makedirs(out_dir)
    except:
        print('Error creating output folder, skip image')
        return;
    
    ## Save image
    try:
        imageio.imsave(f_out, mat_out.astype(np.uint8))    
    except:
        print('Error writing output image, skip image')
        return;
    

#### Read list of images

In [3]:
## Find all images
indir = '../inputs/'
f_all_mb = os.path.join(indir, 'data_small', 'multiband', '*/*.tif')
in_img_list = glob(f_all_mb)

## Save to dataframe
df_in = pd.DataFrame(data = in_img_list, columns = ['FileName'])

In [4]:
df_in.head()

,FileName
0,../inputs/data_small/multiband/Residential/Res...
1,../inputs/data_small/multiband/Residential/Res...
2,../inputs/data_small/multiband/Residential/Res...
3,../inputs/data_small/multiband/Residential/Res...
4,../inputs/data_small/multiband/Residential/Res...


In [5]:
df_in.loc[0].FileName

'../inputs/data_small/multiband/Residential/Residential_2589.tif'

In [6]:
## Detect categories
df_in['Category'] = df_in.FileName.str.split(os.sep).str[4]
df_in.groupby('Category').count()

,FileName
Category,
AnnualCrop,50
Industrial,50
Residential,50


In [7]:
## Detect image names
df_in['ImageName'] = df_in.FileName.str.split(os.sep).str[5].str.replace('.tif', '', regex = False)
df_in = df_in[['ImageName', 'Category', 'FileName']]
df_in.head()

,ImageName,Category,FileName
0,Residential_2589,Residential,../inputs/data_small/multiband/Residential/Res...
1,Residential_1929,Residential,../inputs/data_small/multiband/Residential/Res...
2,Residential_580,Residential,../inputs/data_small/multiband/Residential/Res...
3,Residential_1543,Residential,../inputs/data_small/multiband/Residential/Res...
4,Residential_461,Residential,../inputs/data_small/multiband/Residential/Res...


#### Extract RGB bands from all images

In [9]:
outdir = '../outputs/bands_rgb'

for i, tmp_ind in enumerate(df_in.index):
    tmp_name = df_in.loc[tmp_ind, 'ImageName']
    tmp_cat = df_in.loc[tmp_ind, 'Category']
    tmp_file = df_in.loc[tmp_ind, 'FileName']
    print('Extracting RGB bands from image ' + str(i) + ' : ' + tmp_name)
    f_out = os.path.join(outdir, tmp_cat, tmp_name + '_my_rgb.png')
    select_rgb_bands_from_multiband(tmp_file, f_out)
    
    ## input('Enter a key to continue')   ## For testing the initial run

Extracting RGB bands from image 0 : Residential_2589
Extracting RGB bands from image 1 : Residential_1929
Extracting RGB bands from image 2 : Residential_580
Extracting RGB bands from image 3 : Residential_1543
Extracting RGB bands from image 4 : Residential_461
Extracting RGB bands from image 5 : Residential_339
Extracting RGB bands from image 6 : Residential_795
Extracting RGB bands from image 7 : Residential_1662
Extracting RGB bands from image 8 : Residential_2218
Extracting RGB bands from image 9 : Residential_1982
Extracting RGB bands from image 10 : Residential_1410
Extracting RGB bands from image 11 : Residential_1503
Extracting RGB bands from image 12 : Residential_171
Extracting RGB bands from image 13 : Residential_126
Extracting RGB bands from image 14 : Residential_1715
Extracting RGB bands from image 15 : Residential_2869
Extracting RGB bands from image 16 : Residential_759
Extracting RGB bands from image 17 : Residential_829
Extracting RGB bands from image 18 : Residenti